3. Flow Data

For this exercise we will use the CTU-13 dataset, capture 51.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import os
import sys
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)
from global_helper import *
#from helper import *
from itertools import product

Load data (Note: Right now I manually replace the header with the header from capture 43 because otherwise the reader doesn't work, find a fix for this some time)



In [3]:
#%%time

# Read file and preprocess columns (takes ~1.5min)
# Only loads the traffic associated with the provided IP address.
df = load_data("../data/capture20110818.pcap.netflow.labeled")


In [4]:
df.describe(percentiles=[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]) 

,duration,tos,packets,bytes,flows
count,5.180850e+06,5.180850e+06,5.180850e+06,5.180850e+06,5180850.0
mean,5.043371e-01,5.134042e-02,1.226458e+01,9.342981e+03,1.0
std,1.221683e+00,3.106234e+00,2.640554e+02,3.286442e+05,0.0
min,0.000000e+00,0.000000e+00,1.000000e+00,6.000000e+01,1.0
10%,0.000000e+00,0.000000e+00,1.000000e+00,6.000000e+01,1.0
20%,0.000000e+00,0.000000e+00,1.000000e+00,6.800000e+01,1.0
30%,0.000000e+00,0.000000e+00,1.000000e+00,7.900000e+01,1.0
40%,0.000000e+00,0.000000e+00,1.000000e+00,1.110000e+02,1.0
50%,0.000000e+00,0.000000e+00,1.000000e+00,1.330000e+02,1.0
60%,0.000000e+00,0.000000e+00,1.000000e+00,1.810000e+02,1.0


In [5]:
botnet = df[df['label'] == 'Botnet']
#display(botnet[:20])
botnet.describe()

,duration,tos,packets,bytes,flows
count,323441.000000,323441.0,323441.000000,323441.000000,323441.0
mean,0.593597,0.0,11.345151,10365.959093,1.0
std,1.458396,0.0,80.639121,66730.301757,0.0
min,0.000000,0.0,1.000000,60.000000,1.0
25%,0.000000,0.0,1.000000,1066.000000,1.0
50%,0.000000,0.0,1.000000,1066.000000,1.0
75%,0.000000,0.0,1.000000,1066.000000,1.0
max,22.581000,0.0,1002.000000,788743.000000,1.0


In [6]:
legitimate = df[df['label'] == 'LEGITIMATE']
#display(legitimate[:20])
legitimate.describe()

,duration,tos,packets,bytes,flows
count,321917.000000,321917.0,321917.000000,3.219170e+05,321917.0
mean,0.663173,0.0,23.687808,1.659396e+04,1.0
std,1.336245,0.0,154.185725,1.771371e+05,0.0
min,0.000000,0.0,1.000000,6.000000e+01,1.0
25%,0.000000,0.0,1.000000,6.600000e+01,1.0
50%,0.024000,0.0,2.000000,2.970000e+02,1.0
75%,0.386000,0.0,7.000000,2.046000e+03,1.0
max,4.999000,0.0,16937.000000,2.497247e+07,1.0


Here we discretize the data, using #qcut.

In [28]:
df_actual = df[df['label'] != 'Background']
discretized = pd.qcut(df_actual['packets'], 5, duplicates='drop')
#display(discretized[:10])
#display(discretized.value_counts())

df_actual = df_actual[['label', 'protocol']]
df_actual = pd.concat([df_actual, discretized], axis=1)

display(df_actual[:10])


,label,protocol,packets
2011-08-18 10:19:13.347,LEGITIMATE,TCP,"(5.0, 16937.0]"
2011-08-18 10:19:13.392,LEGITIMATE,TCP,"(0.999, 2.0]"
2011-08-18 10:19:13.411,LEGITIMATE,TCP,"(5.0, 16937.0]"
2011-08-18 10:19:13.460,LEGITIMATE,TCP,"(5.0, 16937.0]"
2011-08-18 10:19:13.486,LEGITIMATE,TCP,"(0.999, 2.0]"
2011-08-18 10:19:13.689,LEGITIMATE,TCP,"(5.0, 16937.0]"
2011-08-18 10:19:13.848,LEGITIMATE,TCP,"(0.999, 2.0]"
2011-08-18 10:19:13.858,LEGITIMATE,TCP,"(2.0, 5.0]"
2011-08-18 10:19:14.067,LEGITIMATE,TCP,"(0.999, 2.0]"
2011-08-18 10:19:14.134,LEGITIMATE,TCP,"(0.999, 2.0]"


In [29]:
botnet_discretized = df_actual[df_actual['label'] == 'Botnet']
botnet_discretized[:10]

,label,protocol,packets
2011-08-18 10:20:40.906,Botnet,TCP,"(0.999, 2.0]"
2011-08-18 10:37:58.448,Botnet,UDP,"(0.999, 2.0]"
2011-08-18 10:37:58.448,Botnet,UDP,"(0.999, 2.0]"
2011-08-18 10:38:00.695,Botnet,UDP,"(0.999, 2.0]"
2011-08-18 10:38:00.705,Botnet,UDP,"(0.999, 2.0]"
2011-08-18 10:38:00.706,Botnet,TCP,"(2.0, 5.0]"
2011-08-18 10:38:00.714,Botnet,TCP,"(2.0, 5.0]"
2011-08-18 10:38:03.255,Botnet,UDP,"(0.999, 2.0]"
2011-08-18 10:38:03.266,Botnet,UDP,"(0.999, 2.0]"
2011-08-18 10:38:03.267,Botnet,UDP,"(0.999, 2.0]"


In [30]:
legitimate_discretized = df_actual[df_actual['label'] == 'LEGITIMATE']
legitimate_discretized[:10]

,label,protocol,packets
2011-08-18 10:19:13.347,LEGITIMATE,TCP,"(5.0, 16937.0]"
2011-08-18 10:19:13.392,LEGITIMATE,TCP,"(0.999, 2.0]"
2011-08-18 10:19:13.411,LEGITIMATE,TCP,"(5.0, 16937.0]"
2011-08-18 10:19:13.460,LEGITIMATE,TCP,"(5.0, 16937.0]"
2011-08-18 10:19:13.486,LEGITIMATE,TCP,"(0.999, 2.0]"
2011-08-18 10:19:13.689,LEGITIMATE,TCP,"(5.0, 16937.0]"
2011-08-18 10:19:13.848,LEGITIMATE,TCP,"(0.999, 2.0]"
2011-08-18 10:19:13.858,LEGITIMATE,TCP,"(2.0, 5.0]"
2011-08-18 10:19:14.067,LEGITIMATE,TCP,"(0.999, 2.0]"
2011-08-18 10:19:14.134,LEGITIMATE,TCP,"(0.999, 2.0]"


TO DO: Make heatmaps of the last two things and there you go!